<a href="https://colab.research.google.com/github/NongNNew/Project_499/blob/main/Project_499_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

In [33]:
!git clone https://github.com/NongNNew/Project_499.git

fatal: destination path 'Project_499' already exists and is not an empty directory.


In [34]:
import pandas as pd
from glob import glob
import librosa
import librosa.display

In [35]:
fruit_veget = ['กระชาย','กระท้อน','กระเทียม1','กระเทียม2','กระเพรา','กล้วยน้ำว้า','กล้วยหอม',
               'ข้าวโพด','ไข่น้ำ',
               'ตะขบไทย','ตะไคร้',
               'ถั่วฝักยาว','ถั่วลันเตา','ถั่วลิสง',
               'ทับทิม',
               'น้อยหน่า','น้ำเต้า',
               'ผักกระเฉด','ผักกุยช่าย',
               'บวบ','ใบชะพลู1','ใบชะพลู2','ใบบัวบก','ใบแมงลัก','ใบยอ',
               'พริกขี้หนู','พริกสด',
               'ฝรั่ง',
               'พุทรา',
               'มะปราง','มะพลับ','มะละกอ',
               'สับปะรด']

source = {'audio_time':[],
          'sampling_rate':[],
          'feature_extraction':[],
          'label':[],
          'description':[]}

In [36]:
for types in fruit_veget:
    audio_files = glob('/content/Project_499/Record_audio/' + str(types) + '/*.wav')

    for audio in audio_files:
        y,sr = librosa.load(audio,duration=6,offset=0)
        source['description'].append(str(types))
        source['audio_time'].append(y)
        source['sampling_rate'].append(sr)
        source['feature_extraction'].append(librosa.feature.mfcc(y,sr,n_mfcc=13))
        source['label'].append(fruit_veget.index(types))

In [37]:
import re

for i in range(len(source['description'])):
    source['description'][i] = re.findall('[\u0E00-\u0E7F]+',source['description'][i])[0]

In [38]:
pd.DataFrame.from_dict(source)

,audio_time,sampling_rate,feature_extraction,label,description
0,"[-0.0002593994, -0.00022888184, -0.0002593994,...",22050,"[[-639.25616, -639.47345, -640.0064, -640.5056...",0,กระชาย
1,"[1.5258789e-05, -3.0517578e-05, 4.5776367e-05,...",22050,"[[-624.1429, -624.4267, -624.8286, -624.58374,...",0,กระชาย
2,"[-0.004928589, -0.0049591064, -0.0048980713, -...",22050,"[[-634.622, -639.0867, -654.00525, -659.99115,...",0,กระชาย
3,"[1.5258789e-05, -3.0517578e-05, 6.1035156e-05,...",22050,"[[-635.46277, -636.8741, -637.42834, -636.0504...",0,กระชาย
4,"[-0.0001373291, -0.00018310547, -4.5776367e-05...",22050,"[[-655.651, -653.5817, -654.66907, -655.0482, ...",0,กระชาย
...,...,...,...,...,...
654,"[-0.00018310547, -0.0002593994, -0.00022888184...",22050,"[[-662.2502, -662.54706, -661.8858, -660.38995...",32,สับปะรด
655,"[-0.00019836426, -0.0002593994, -7.6293945e-05...",22050,"[[-655.92065, -656.85034, -655.4548, -653.7831...",32,สับปะรด
656,"[-0.00016784668, -0.00015258789, -0.0001983642...",22050,"[[-668.4974, -670.4946, -672.10724, -671.17737...",32,สับปะรด
657,"[-0.00018310547, -0.0002593994, -0.00021362305...",22050,"[[-670.37274, -672.49, -674.93616, -677.3624, ...",32,สับปะรด


In [39]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np
from sklearn.model_selection import train_test_split

X = np.array(source['feature_extraction'])
y = np.array(source['label'])

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 5, input_shape=(13, 259, 1), activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(64, 5, activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(len(np.unique(source['label'])), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [40]:
# # Cumulative Frequency of label y_test
# from collections import Counter
# Counter(y_test)

# Showing label are used in testing model. 
np.unique(y_test)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32])

In [41]:
model.fit(X_train[:,:,:,None], y_train, epochs=100)

Epoch 1/100
16/16 [==============================] - 1s 10ms/step - loss: 30.3935 - accuracy: 0.0344
Epoch 2/100
16/16 [==============================] - 0s 10ms/step - loss: 3.4526 - accuracy: 0.0405
Epoch 3/100
16/16 [==============================] - 0s 8ms/step - loss: 3.2684 - accuracy: 0.0769
Epoch 4/100
16/16 [==============================] - 0s 8ms/step - loss: 2.7816 - accuracy: 0.2267
Epoch 5/100
16/16 [==============================] - 0s 8ms/step - loss: 2.2942 - accuracy: 0.3644
Epoch 6/100
16/16 [==============================] - 0s 8ms/step - loss: 1.8968 - accuracy: 0.4332
Epoch 7/100
16/16 [==============================] - 0s 8ms/step - loss: 1.6806 - accuracy: 0.5202
Epoch 8/100
16/16 [==============================] - 0s 8ms/step - loss: 1.3753 - accuracy: 0.5789
Epoch 9/100
16/16 [==============================] - 0s 8ms/step - loss: 1.1587 - accuracy: 0.6700
Epoch 10/100
16/16 [==============================] - 0s 8ms/step - loss: 0.9482 - accuracy: 0.7065
Epoch 

In [42]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 9, 255, 64)        1664      
_________________________________________________________________
dropout_12 (Dropout)         (None, 9, 255, 64)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 4, 127, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 4, 127, 64)        102464    
_________________________________________________________________
dropout_13 (Dropout)         (None, 4, 127, 64)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 2, 63, 64)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 8064)             

In [43]:
model.evaluate(X_test[:,:,:,None],y_test)

6/6 [==============================] - 0s 4ms/step - loss: 0.2086 - accuracy: 0.9636


[0.2085624635219574, 0.9636363387107849]

In [44]:
from sklearn.metrics import classification_report

predicted_classes = np.argmax(np.round(model.predict(X_test[:,:,:,None])),axis=1)
target_names = ["Class {}".format(i) for i in range(len(np.unique(source['label'])))]
print(classification_report(y_test, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.86      1.00      0.92         6
     Class 1       1.00      1.00      1.00         3
     Class 2       1.00      1.00      1.00         2
     Class 3       0.89      0.89      0.89         9
     Class 4       1.00      1.00      1.00         8
     Class 5       1.00      1.00      1.00         3
     Class 6       0.86      1.00      0.92         6
     Class 7       0.86      1.00      0.92         6
     Class 8       1.00      1.00      1.00         8
     Class 9       1.00      1.00      1.00         4
    Class 10       1.00      1.00      1.00         3
    Class 11       1.00      1.00      1.00         6
    Class 12       1.00      1.00      1.00         4
    Class 13       0.75      1.00      0.86         3
    Class 14       1.00      1.00      1.00         4
    Class 15       1.00      1.00      1.00         4
    Class 16       0.80      1.00      0.89         4
    Class 17       1.00    